追従実験考えてみる
1. ユーザーをsimhashでいい感じにコホートに分解できるのか
2. コホートの性質の分析
3. コホートからリバースエンジニアリングできるか？

1. ユーザーをsimashでいい感じにコホートに分解できるのか  
シンプルなoneHotで試してみる．

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
d = 1000
n = 10000
root_cohort_num = 10 # 大カテゴリ， Web上の履歴に大きく影響を及ぼす(性別とか年齢とか)
p_root_cohort = 0.01
week_cohort_num = 100 # 少カテゴリ， 小さい確率でそれっぽい行動をする(個人端末で仕事のこと調べるとか)
p_week_cohort = 0.005
p_noize = 0.001

In [ ]:
cohort_roots = [np.random.binomial(10, p_root_cohort, d) for i in range(root_cohort_num)]
cohort_week = [np.random.binomial(10, p_week_cohort, d) for i in range(week_cohort_num)]
result = []
for i in range(0, n):
  # 誤差乗せる
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  result.append(np.where(cohort_roots[root_idx] + cohort_week[week_idx] + np.random.binomial(10, p_noize, d), 1, 0) )
  #誤差 ０.01だとそこそこブレる， まぁぶれたならそこが大事だからちゃんと反映されるねという話はある
  #よく考えたらカラムはドメインなので， 0.01 * dした数だけ違うドメインに飛んでたらそれは違うクラスタだわ

In [ ]:
weights = [np.random.randint(-1, 2, d) for i in range(100)] # 適当に100bitでやってみる

In [ ]:
vecs = []
for i in range(n):
  vec = []
  for weight in weights:
    vec.append(1 if np.dot(result[i], weight) >= 0 else 0)
  vecs.append(vec)

In [ ]:
bites = [bidnizer(each) for each in vecs]

In [ ]:
def bidnizer(tgt : list):
  out = 0
  for each in tgt:
    out = (out << 1) | each
  return out

In [ ]:
cohort_id = []
week_cohort_id = []
for i in range(0, n):
  root_idx = i % root_cohort_num
  week_idx = i % week_cohort_num
  cohort_id.append(root_idx)
  week_cohort_id.append(week_idx)

In [ ]:
tmpDf = pd.DataFrame(list(zip(bites, cohort_id, week_cohort_id)), columns = ["bite", "cohort_id", "week_cohort_id"])

In [ ]:
sTmpDf = tmpDf.sort_values("bite")

In [ ]:
pd.set_option("max_rows", 100000)

In [ ]:
sTmpDf
# cohort_idの並びを見るとそこそこできている…のか？
# 弱いコホートに関してもまぁある程度集約できてる
# ノイズによって変わるけど十分大きくするとメモリが足りなくなって面倒なのでココで終わる

,bite,cohort_id,week_cohort_id
1524,5753579242081272212193691880,4,24
1395,5803275707081562908607415134,5,95
1165,5805486070428070863192116216,5,65
4515,6132111056449468543274224157,5,15
1904,6362913844763889209720622312,4,4
1304,6362916205947977200347565800,4,4
1424,6989058909982333302457157865,4,24
7124,6989080183689936330143746280,4,24
4715,7293294267432175200067814040,5,15
6663,11950673157008433282809112778,3,63


In [ ]:
sTmpDf.loc[(sTmpDf.cohort_id == 69) & (sTmpDf.week_cohort_id == 869)]

,bite,cohort_id,week_cohort_id
869,17681930828797471918457330534,69,869
4869,17759594865677998392051997028,69,869
1869,18300900848440179648093985126,69,869
7869,18301203079967123306164754790,69,869
5869,18301203079967140899425589604,69,869
6869,18306038783245581961376274788,69,869
9869,335213853137024473817779509604,69,869
3869,335218688840239952291259453798,69,869
2869,335223524543581390851177159012,69,869
8869,374837605798409968285004044646,69,869


KeyError: ignored